In [ ]:
import pandas as pd
import urllib.request
import numpy as np
from IPython.display import display
import os

# Data Collection #

For our analysis of cancer prediction using gene mutation and clinical data from patients, we will gather data from multiple sources. First we obtain the gene mutation data from the $cancer.gov$ website. Next we get the patient clinical data also from the $cancer.gov$ website. In our notebook, we store this data locally so that it does not have to downloaded if the notebook kernel is restarted and run multiple times.

In [ ]:
# This downloads a 753 MB mutations gzip file.  This will take about 1-5 mins depending on your
# connection speed.
filename = "pancancer_mutations.maf.gz"
if os.path.isfile(filename):
    print("Skipping download, as file %s is present" %(filename))
else:
    print('Downloading mutation data. 753 MB ...')
    url = 'http://api.gdc.cancer.gov/data/1c8cfe5f-e52d-41ba-94da-f15ea1337efc'  
    urllib.request.urlretrieve(url, './pancancer_mutations.maf.gz')  
print("done.")

In [ ]:
# This downloads an 18 MB clinical data file
filename = "pancancer_clinical.txt"
if os.path.isfile(filename):
    print("Skipping download, as file %s is present" %(filename))
else:
    print('Downloading clinical data ...')
    url = 'http://api.gdc.cancer.gov/data/0fc78496-818b-4896-bd83-52db1f533c5c'  
    urllib.request.urlretrieve(url, './pancancer_clinical.txt')  
print("done.")

## Loading Gene Mutation Data ##

Here we read the gene mutation data. This data file contains many columns, but after careful curation, we have decided to consider the following colums:

1. **Tumor_Sample_Barcode**: this contains the barcode with the first 12 characters identifying the patient
2. **Hugo_Symbol**: this is the actual gene that has been mutated (for e.g. TACC2, JAKMIP3, PANX3)
3. **Chromosome**: the chromosome tells us the location of the gene where the mutation is seen
4. **Variant_Classification**: this indicates the kind of mutation seen in the gene
5. **Variant_Type**: this indicates the cancer type that we are trying to predict

In [ ]:
# Load the mutations dataframe
print('Loading mutations dataframe ...')
mutations = pd.read_csv("pancancer_mutations.maf.gz", compression='gzip',
                        sep='\t',
                        usecols=['Tumor_Sample_Barcode','Hugo_Symbol', 
                                 'Variant_Classification', 'Variant_Type',
                                'Chromosome'])
print("done.")
print("Mutations count", mutations.Tumor_Sample_Barcode.count())

In [ ]:
mutations.head()

In [ ]:
len(mutations['Hugo_Symbol'].unique())

In [ ]:
# Set mutations index
mutations['row'] = np.arange(len(mutations))
mutations.set_index('row')

The actual cancer type for each of the gene mutations above is kept in a different file. The entries are located at corresponding rows, so we use that to merge the cancer type from the labels.txt file.

In [ ]:
# Read the cancertypes.  These are the matching labels for the mutations file
print("Loading labels dataframe ...")
labels = pd.read_csv("labels.txt", sep='\t')
labels.columns = ["cancer_type"]
labels['row'] = np.arange(len(labels))
labels.set_index('row')
print('Labels count', labels.cancer_type.count())

As we can see, there are **3600963** data points in both the gene dataframe and the cancer label dataframe. So now we merge the label information into the main mutations dataframe. The idea is that when we bring in the patient information, we will go from a _row for each gene_ to a _row for each patient_, with all the gene mutation data for that patient added as columns. With this, the cancer type (label) will also gets included as a column. 

In [ ]:
# Merge the labels with the mutations
mutations = mutations.merge(labels, left_on='row', right_on='row')
display(mutations.head())

## Loading Patient Data##

Here we load the clinical data. This is data for patients for whom we collected the gene mutation data above. The patients are identified by $bcr\_patient\_barcode$. We will use this field to populate the gene mutation data from the dataframe above in the table we are about to read. The clinical data has patient information such as age, gender etc.

In [ ]:
# Load the clinical data
print('Loading clinical dataframe ...')
clinical = pd.read_csv("pancancer_clinical.txt", sep='\t',
                        usecols=['bcr_patient_barcode', 'gender', 
                                 'vital_status', 'days_to_birth', 'days_to_death', 
                                 'age_at_initial_pathologic_diagnosis', 'pathologic_stage', 
                                 'height', 'weight'])

print('Clinical count', clinical.bcr_patient_barcode.count())
display(clinical.head())

## Creating Merged Data ##

Now that we have both gene and cancer data in one dataframe, and the patient clinical data in another dataframe, we will use the **bcr_patient_barcode** to merge these into a single table. With this, we can drop the Tumor_Sample_Barcode column, since it has served its purpose. Looking at the data, it seems like some patient data is missing from the gene data. Simultaneously, some data in the gene dataframe does not have corresponding clinical data. Hence our merged dataframe size will be lower than the original mutations dataframe size.

In [ ]:
clinical['bcr_patient_barcode'].isnull().values.any()

In [ ]:
missing_count = 0
gene_barcode_set = set(mutations.bcr_patient_barcode.unique())
for bcr in gene_barcode_set:
    if bcr not in set(clinical.bcr_patient_barcode.unique()):
        missing_count += 1
print("%d patients with gene data missing in clinical data" %missing_count)

In [ ]:
# Get the patient barcode.  This is what we will use to join the mutations to the clinical data
def parsePatientBarcode(tumor_sample_barcode):
        return tumor_sample_barcode[0:12]

mutations['bcr_patient_barcode'] = mutations['Tumor_Sample_Barcode'].apply(parsePatientBarcode)
mutations = mutations.drop(['Tumor_Sample_Barcode'], axis=1)
mutations.head()

In [ ]:
merged = mutations.merge(clinical, left_on='bcr_patient_barcode', right_on='bcr_patient_barcode')
display(merged.head())
merged.bcr_patient_barcode.count()
print('Merged mutations count', merged.bcr_patient_barcode.count())

In [ ]:
# Write out new merged dataframe as csv file
print("Writing new csv file with merged mutations, clinical, and cancer types ...")
merged.to_csv("pancancer_mutations_merged.csv")
print("done.")